In [233]:
from hcve_lib.feature_importance import show_permutation_importance
from hcve_lib.splitting import get_lco_splits
from hcve_lib.utils import notebook_init

notebook_init()

from mlflow import set_tracking_uri

%autoreload 2

from deps.common import get_variables_cached
from hcve_lib.tracking import search_latest_root_completed_run
from hcve_lib.feature_importance import get_permutation_importance
from deps.pipelines import get_pipelines, LCO_GB_HYPERPARAMETERS
from utils import compute_standard_metrics
from hcve_lib.cv import cross_validate
from plotly.graph_objs import Figure
from pandas import Series
from hcve_lib.utils import transpose_dict
from plotly import express as px
import plotly.graph_objs as go
from hcve_lib.functional import t
from pandas import DataFrame
from functools import partial
from hcve_lib.utils import partial2
from hcve_lib.evaluation_functions import compute_metrics_folds, c_index

set_tracking_uri('http://localhost:5000')

data, metadata, X, y = get_variables_cached()

In [137]:
run, result = search_latest_root_completed_run('optimized_lm', 'coxnet')

fig = show_permutation_importance(
    result[('ASCOT', 'FLEMENGHO')],
    X,
    y,
    metadata,
    train_importance=True,
    n_repeats=10,
    limit=10,
)

In [165]:
run, result = search_latest_root_completed_run('optimized_lm', 'gb')
fig = show_permutation_importance(
    result[('ASCOT', 'HVC')],
    X,
    y,
    metadata,
    train_importance=True,
    n_repeats=10,
    limit=10,
)

In [158]:

importance = get_permutation_importance(
    result[('ASCOT', 'FLEMENGHO')],
    X,
    y,
)

In [ ]:
current_method = get_pipelines()['gb']

X_actual = X.copy()
results_per_n_features = {}

for feature in reversed(importance.index):
    print(feature)
    results_per_n_features[len(X_actual.columns)] = cross_validate(
        X_actual,
        y,
        current_method.get_estimator,
        current_method.predict,
        get_lco_splits(X_actual, y, data=data),
        n_jobs=-1,
        split_hyperparameters=LCO_GB_HYPERPARAMETERS,
    )

    X_actual = X_actual.drop(columns=feature)

metrics_per_n_features = {
    n_features: {split: c_index(result, X, y) for split, result in results.items()} for n_features, results in results_per_n_features.items()
}

c_statistic_per_n_features = {
    n_features: compute_standard_metrics(X, y, result) for n_features, result in results_per_n_features.items()
}

CI
TG
DRK
CREA
HR
SMK
HCV
HCAD
HPAD


In [232]:
fig = px.line(DataFrame(metrics_per_n_features).T)
fig.for_each_trace(lambda line: fig.add_annotation(
    x=line.x[0], y=line.y[0], text=line.name,
    font_color=line.line.color, ax=0, ay=0, xanchor="left", showarrow=False
))
fig.update_layout(
    showlegend=False,
    xaxis_title="n features",
    yaxis_title='c-statistic',
)

In [ ]:
fig = px.line(DataFrame(metrics_per_n_features).T)
fig.for_each_trace(lambda line: fig.add_annotation(
    x=line.x[0], y=line.y[0], text=line.name,
    font_color=line.line.color, ax=0, ay=0, xanchor="left", showarrow=False
))
fig.update_layout(
    showlegend=False,
    xaxis_title="n features",
    yaxis_title='c-statistic',
)
